In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import missingno as msno

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report, precision_recall_curve
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, RandomizedSearchCV

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense
#from sklearn.model_selection import GridSearchCV
#from sklearn.model_selection import RandomizedSearchCV
from kerastuner.tuners import RandomSearch


# Objective:

Don’t deprive me learning from your valuable feedback.Advance thanks for your time.

# Let's Understand the problem


A stroke occurs when part of the brain loses its blood supply and stops working. This causes the part of the body that the injured brain controls to stop working.
A stroke also is called a cerebrovascular accident, CVA, or "brain attack."

The types of strokes include:

Ischemic stroke (part of the brain loses blood flow)

Hemorrhagic stroke (bleeding occurs within the brain)

Transient ischemic attack, TIA, or mini-stroke (The stroke symptoms resolve within minutes, but may take up to 24 hours on their own without treatment. This is a warning sign that a stroke may occur in the near future.)



# Causes of Stroke:

Age. Stroke risk increases with age. Three-quarters of strokes occur in people ages 65 and older.

Geography. The highest U.S. death rates from stroke occur in the southeastern United States.

Gender. Men are more likely than women to have a stroke.

Certain lifestyle factors and conditions also increase the risk for stroke. The most important of these include:

High blood pressure

Diabetes

Heart disease (such as atrial fibrillation)

Previous stroke or transient ischemic attack

Cigarette smoking

Additional risk factors include:

Physical inactivity

Overweight or obesity

High cholesterol

Sickle cell disease

Drinking too much alcohol

Family history of stroke

Drug abuse

Genetic conditions, such as blood-clotting or vascular disorders (for example, Factor V Leiden or CADASIL)

Certain medications (such as hormonal birth control pills)

Being pregnant

Menopause

Lesser risk factors include:

Head and neck injuries

Recent viral or bacterial infections

In [ ]:
df = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().mean()

In [ ]:
msno.bar(df)

Here we are seeing that 7 Categorical features.So let's start the EDA with those features.



In [ ]:
#plt.rcParams['figure.figsize'] = (20.0, 10.0)
#plt.rcParams['font.family'] = "serif"
#plt.rcParams['font.family'] = "cursive"
bg_color = (.23,.34,.29)
sns.set(rc={"font.style":"normal",
            "axes.facecolor":bg_color,
            "figure.facecolor":bg_color,
            "text.color":"white",
            "xtick.color":"white",
            "ytick.color":"white",
            "axes.labelcolor":"white",
            "axes.grid":False,
            'axes.labelsize':30,
            'figure.figsize':(20.0, 10.0),
            'xtick.labelsize':25,
            'ytick.labelsize':20})


f, axes = plt.subplots(2,4, figsize=(30,25))

ax = sns.countplot(data=df,
                  x = 'gender',
                  hue = 'stroke',
             
                  edgecolor=(0,0,0),
                  linewidth=5,
                  palette="prism",ax=axes[0,0]
                  )
ax.set_xticklabels(ax.get_xticklabels(),rotation=30)
for rect in ax.patches:
    ax.text (rect.get_x() + rect.get_width()  / 2,rect.get_height()+ 0.25,rect.get_height(),horizontalalignment='center', fontsize = 20.0)

ax1 = sns.countplot(data=df,
                  x = 'hypertension',
                  hue = 'stroke',
             
                  edgecolor=(0,0,0),
                  linewidth=5,
                  palette="cool",ax=axes[0,1]
                  )
for rect in ax1.patches:
    ax1.text (rect.get_x() + rect.get_width()  / 2,rect.get_height()+ 0.25,rect.get_height(),horizontalalignment='center', fontsize = 20.0)

ax2 = sns.countplot(data=df,
                  x = 'heart_disease',
                  hue = 'stroke',
             
                  edgecolor=(0,0,0),
                  linewidth=5,
                  palette="Accent",ax=axes[0,2]
                  )
for rect in ax2.patches:
    ax2.text (rect.get_x() + rect.get_width()  / 2,rect.get_height()+ 0.25,rect.get_height(),horizontalalignment='center', fontsize = 20.0)
   
ax3 = sns.countplot(data=df,
                  x = 'ever_married',
                  hue = 'stroke',
             
                  edgecolor=(0,0,0),
                  linewidth=5,
                  palette="Accent_r",ax=axes[0,3]
                  )
for rect in ax3.patches:
    ax3.text (rect.get_x() + rect.get_width()  / 2,rect.get_height()+ 0.25,rect.get_height(),horizontalalignment='center', fontsize = 20.0)
   

ax4 = sns.countplot(data=df,
                  x = 'work_type',
                  hue = 'stroke',
             
                  edgecolor=(0,0,0),
                  linewidth=5,
                  palette="twilight_shifted_r",ax=axes[1,0]
                  )
ax4.set_xticklabels(ax4.get_xticklabels(),rotation=30)
for rect in ax4.patches:
    ax4.text (rect.get_x() + rect.get_width()  / 2,rect.get_height()+ 0.25,rect.get_height(),horizontalalignment='center', fontsize = 20.0)

ax5 = sns.countplot(data=df,
                  x = 'Residence_type',
                  hue = 'stroke',
             
                  edgecolor=(0,0,0),
                  linewidth=5,
                  palette="twilight_shifted",ax=axes[1,1]
                  )
for rect in ax5.patches:
    ax5.text (rect.get_x() + rect.get_width()  / 2,rect.get_height()+ 0.25,rect.get_height(),horizontalalignment='center', fontsize = 20.0)

ax6 = sns.countplot(data=df,
                  x = 'smoking_status',
                  hue = 'stroke',
             
                  edgecolor=(0,0,0),
                  linewidth=5,
                  palette="Accent_r",ax=axes[1,2]
                  )
ax6.set_xticklabels(ax6.get_xticklabels(),rotation=30)
for rect in ax6.patches:
    ax6.text (rect.get_x() + rect.get_width()  / 2,rect.get_height()+ 0.25,rect.get_height(),horizontalalignment='center', fontsize = 20.0)
   
f.suptitle('Features distribution based on Stroke ',fontsize=40)

Here we cann't compare category of each features among them, because they are not same in number.So we have to calculate the ratio.



Make Observations:
    
    Gender:
        
1. Male = (108+2007) = .054 and Female = (141/2853) = .045 

2. Male has more porne to Stroke

    Hypertension:
        
1. No =((183/4429)) =.0413 and Yes = 66/432 = .152

2. This clearly shows that people with hypertension are more porne to stroke.054

    Heart Disease:
        
1. No = ((202/4632)) = .044 and Yes = (47./229) = .20

2. This also shows people having heart disease are more porne to stroke.054

    Married:
        
1. Yes = (220/3133) = .07 NO = (29/1728) = 0.016

2. It indaicates Unmarried people are safer then then married.

In [ ]:
#Let's analyze the Numerical Columns.

sns.set_style(style = 'darkgrid')
f, axes = plt.subplots(1,3, figsize = (15,5))

sns.kdeplot(df['age'],shade = True ,ax=axes[0])
sns.kdeplot(df['avg_glucose_level'],shade = True ,ax=axes[1])
sns.kdeplot(df['bmi'],shade = True ,ax=axes[2])
f.suptitle('Numerical Features distribution',fontsize=40)



In [ ]:
g = sns.kdeplot(df['age'][(df["stroke"] == 0)] , color="Red", shade = True)
g = sns.kdeplot(df['age'][(df["stroke"] == 1)], ax =g, color="Blue", shade= True)
g.set_xlabel('Age Distribution')
g.set_ylabel("Frequency")
g = g.legend(["No,","Yes"],fontsize = 30)

Observations:
    
1.People do have stroke mostly are Old and ranging from around 60 to 100 

In [ ]:
g = sns.kdeplot(df['avg_glucose_level'][(df["stroke"] == 0)] , color="Red", shade = True)
g = sns.kdeplot(df['avg_glucose_level'][(df["stroke"] == 1)], ax =g, color="Green", shade= True)
g.set_xlabel('Avg_glucose_level Distribution')
g.set_ylabel("Frequency")
g = g.legend(["No,","Yes"],fontsize = 30)

Obserbations:
    
1. People have stroke having avg_glucose_level around 100 and 200.

2. People  don't have stroke having avg_glucose_level around 75.


In [ ]:
sns.set_palette(palette = 'YlGn')
g = sns.kdeplot(df['bmi'][(df["stroke"] == 0)] , color="Red", shade = True)
g = sns.kdeplot(df['bmi'][(df["stroke"] == 1)], ax =g, color="Yellow", shade= True)
g.set_xlabel('BMI Distribution')
g.set_ylabel("Frequency")
g = g.legend(["No,","Yes"],fontsize = 30)

In [ ]:
plt.Figure(figsize = (10,15))
g = sns.FacetGrid(df, col="stroke",height=8, aspect=1,palette = 'blue', sharey = False)
g.map(sns.scatterplot, "age", "bmi")

In [ ]:
plt.Figure(figsize = (10,15))
g = sns.FacetGrid(df, col="stroke",height=8, aspect=1,palette = 'BuPu', sharey = False)
g.map(sns.scatterplot, "age", "avg_glucose_level")

In [ ]:

plt.Figure(figsize = (10,15))
g = sns.FacetGrid(df, col="stroke",hue = 'stroke', height=8, aspect=1,palette = 'BuPu', sharey = False)
g.map(sns.scatterplot, "bmi", "avg_glucose_level")

# Feature Engineering

In [ ]:
df.dropna(inplace = True)

In [ ]:
df = pd.get_dummies(df,
                    columns=['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status'],
                    drop_first=True)

In [ ]:
X = df.drop(['stroke'], axis=1)
y = df['stroke']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20, random_state=42)

# Handling imbalance data using SMOTE based techniques


In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

counter = Counter(y_train)
print('Before',counter)
# oversampling the train dataset using SMOTE
smt = SMOTE()
#X_train, y_train = smt.fit_resample(X_train, y_train)
X_train_sm, y_train_sm = smt.fit_resample(X_train, y_train)

counter = Counter(y_train_sm)
print('After',counter)

In [ ]:
#ADASYN
from imblearn.over_sampling import ADASYN

counter = Counter(y_train)
print('Before',counter)
# oversampling the train dataset using ADASYN
ada = ADASYN(random_state=130)
X_train_ada, y_train_ada = ada.fit_resample(X_train, y_train)

counter = Counter(y_train_ada)
print('After',counter)


In [ ]:
#SMOTE + TOMEK
from imblearn.combine import SMOTETomek

counter = Counter(y_train)
print('Before',counter)
# oversampling the train dataset using SMOTE + Tomek
smtom = SMOTETomek(random_state=139)
X_train_smtom, y_train_smtom = smtom.fit_resample(X_train, y_train)

counter = Counter(y_train_smtom)
print('After',counter)


In [ ]:
# SMOTE+ ENN
from imblearn.combine import SMOTEENN

counter = Counter(y_train)
print('Before',counter)
#oversampling the train dataset using SMOTE + ENN
smenn = SMOTEENN()
X_train_smenn, y_train_smenn = smenn.fit_resample(X_train, y_train)

counter = Counter(y_train_smenn)
print('After',counter)

In [ ]:
#SMOTE + UNDER SAMPLING
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
sm = SMOTE(sampling_strategy = .3)
rus =  RandomUnderSampler(sampling_strategy=.4)

pipeline = Pipeline(steps = [('smote', sm),('under',rus)])

counter = Counter(y_train)
print('Before',counter)
#over and undersampling the train dataset using SMOTE + RandomUnderSampler
X_train_smrus, y_train_smrus = pipeline.fit_resample(X_train, y_train)

counter = Counter(y_train_smrus)
print('After',counter)

# Model Building

In [ ]:
sampled_data = {
    'ACTUAL':[X_train, y_train],
    'SMOTE':[X_train_sm, y_train_sm],
    'ADASYN':[X_train_ada, y_train_ada],
    'SMOTE_TOMEK':[X_train_smtom, y_train_smtom],
    'SMOTE_ENN':[X_train_smenn, y_train_smenn],
    'UNDERSAMPLING': [X_train_smrus, y_train_smrus]
}

In [ ]:
def test_eval(clf_model, X_test, y_test, algo=None, sampling=None):
    # Test set prediction
    y_prob=clf_model.predict_proba(X_test)
    y_pred=clf_model.predict(X_test)

    print('Confusion Matrix')
    print('='*60)
    #plot_confusion_matrix(clf_model, X_test, y_test)  
    #plt.show() 
    print(confusion_matrix(y_test,y_pred),"\n")
    print('Classification Report')
    print('='*60)
    print(classification_report(y_test,y_pred),"\n")
    print('AUC-ROC')
    print('='*60)
    print(roc_auc_score(y_test, y_prob[:,1]))
    
    #x = roc_auc_score(y_test, y_prob[:,1])
    f1 = f1_score(y_test, y_pred, average='binary')
    recall = recall_score(y_test, y_pred, average='binary')
    precision = precision_score(y_test, y_pred, average='binary')
          
    
    return algo,precision,recall,f1,sampling
    

In [ ]:
model_params = {

    'random-forest' : {
        'model' : RandomForestClassifier(),
        'params' : {
             "n_estimators": [5, 10, 15, 20, 25], 
             'max_depth': [i for i in range(5,16,2)],
             'min_samples_split': [2, 5, 10, 15, 20, 50, 100],
             'min_samples_leaf': [1, 2, 5],
             'criterion': ['gini', 'entropy'],
             'max_features': ['log2', 'sqrt', 'auto']

         }
    },
    'logisticregression': {
        'model' : LogisticRegression(),
        'params' : {
            'C' : [1,6,10],
            'penalty': ['l1', 'l2']
        }
    },
    'decision_tree' :{
        'model' :  DecisionTreeClassifier(),
        'params' : {
             'max_depth': [i for i in range(5,16,2)],
             'min_samples_split': [2, 5, 10, 15, 20, 50, 100],
             'min_samples_leaf': [1, 2, 5],
             'criterion': ['gini', 'entropy'],
             'max_features': ['log2', 'sqrt', 'auto']
        }
        
    }
    
}

In [ ]:
cv = StratifiedKFold(n_splits=5, random_state=100, shuffle=True)
output = []
for model , model_hp in model_params.items():
    for resam , data in sampled_data.items():
        clf = RandomizedSearchCV(model_hp['model'], model_hp['params'],cv = cv, scoring='roc_auc', n_jobs=-1 )
        clf.fit(data[0], data[1])
        clf_best = clf.best_estimator_
       
        print(model+' with ' + resam)
        print('='*60)
        output.append(test_eval(clf_best, X_test, y_test, model, resam))

Let's try it with neural network 

In [ ]:
def build_model(hp):
    model = keras.models.Sequential()
    
    model.add(layers.Dense(units=hp.Int('Input_units',min_value = 6, max_value = 12),
                    activation =hp.Choice('input_act', ['relu', 'sigmoid']), input_dim = 17))
    
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid'])))
        
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
         metrics='accuracy')
    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir2',
    project_name='StrokePrediction')

In [ ]:
def train_test(X_train,y_train,X_test, y_test,algo,sampling = None ):
    tuner.search(X_train, y_train,
                 epochs=5,
                 validation_data=(X_test, y_test))
    best_hp = tuner.get_best_hyperparameters()[0]
    model = tuner.hypermodel.build(best_hp)
    model.fit(X_train, y_train, epochs = 5, validation_split = .1 )
    y_p = model.predict(X_test)
    y_pred = []
    for element in y_p:
        if element > .5:
            y_pred.append(1)
        else:
            y_pred.append(0)

    f1 = f1_score(y_test, y_pred, average='binary')
    recall = recall_score(y_test, y_pred, average='binary')
    precision = precision_score(y_test, y_pred, average='binary')

    print(classification_report(y_test,y_pred),"\n")
    
    return algo,precision,recall,f1,sampling

# ACTUALL:

In [ ]:
output.append(train_test(X_train, y_train, X_test, y_test, 'ANN', 'ACTUAL'))

# SMOTE:

In [ ]:
output.append(train_test(X_train_sm, y_train_sm, X_test, y_test, 'ANN', 'SMOTE'))

# ADASYN:

In [ ]:
output.append(train_test(X_train_ada, y_train_ada, X_test, y_test, 'ANN', 'SMOTE'))

# SMOTE + TOMEK:

In [ ]:
output.append(train_test(X_train_smtom, y_train_smtom, X_test, y_test, 'ANN', 'SMOTE_TOMEK'))

# SMOTE + ENN:

In [ ]:
output.append(train_test(X_train_smenn, y_train_smenn, X_test, y_test, 'ANN', 'SMOTE_ENN'))

# UNDER_SAMPLING:

In [ ]:
output.append(train_test(X_train_smrus, y_train_smrus, X_test, y_test, 'ANN', 'UNDERSAMPLING'))

In [ ]:
result = pd.DataFrame(output, columns = ['Model','Precision','Recall', 'F1-score','Resample'])

In [ ]:
result

surprisingly in every Smote technique
in ANN i got alomost same type of
result.Can anyone help me 
in fixing this problem. 

Work in progress.